In [1992]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import numpy as np
data = pd.read_csv("train.csv")
#data
data_file = pd.read_csv("test.csv")
data_result = pd.read_csv("gender_submission.csv")
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [1993]:
#fill null values
data.drop('PassengerId', axis=1, inplace=True)

data['Name'] = data['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
data['Name'] = data['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Name'] = data['Name'].replace('Mlle', 'Miss')
data['Name'] = data['Name'].replace('Ms', 'Miss')
data['Name'] = data['Name'].replace('Mme', 'Mrs')

data['Embarked'] = data['Embarked'].fillna("S")

data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

data['relatives'] = data['SibSp'] + data['Parch']
data.loc[data['relatives'] > 0, 'alone'] = 0
data.loc[data['relatives'] == 0, 'alone'] = 1
data['alone'] = data['alone'].astype(int)

data.loc[ data['Fare'] <= 7.91, 'Fare'] = 0
data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
data.loc[(data['Fare'] > 31) & (data['Fare'] <= 99), 'Fare']   = 3
data.loc[(data['Fare'] > 99) & (data['Fare'] <= 250), 'Fare']   = 4
data.loc[ data['Fare'] > 250, 'Fare'] = 5
data['Fare'] = data['Fare'].astype(int)

data.drop('Ticket', axis=1, inplace=True)

for index,each in enumerate(data["Age"].isnull()):
    if each == True:
        data.loc[index,'Age'] = data[data['Name'] == data["Name"][index]]['Age'].mean()
data['Age'] = data['Age'].astype(int)
data.loc[ data['Age'] <= 11, 'Age'] = 0
data.loc[(data['Age'] > 11) & (data['Age'] <= 18), 'Age'] = 1
data.loc[(data['Age'] > 18) & (data['Age'] <= 22), 'Age'] = 2
data.loc[(data['Age'] > 22) & (data['Age'] <= 27), 'Age'] = 3
data.loc[(data['Age'] > 27) & (data['Age'] <= 33), 'Age'] = 4
data.loc[(data['Age'] > 33) & (data['Age'] <= 40), 'Age'] = 5
data.loc[(data['Age'] > 40) & (data['Age'] <= 66), 'Age'] = 6
data.loc[ data['Age'] > 66, 'Age'] = 6

data['Age_Class']= data['Age']* data['Pclass']


data['Fare_Per_Person'] = data['Fare']/(data['relatives']+1)
data['Fare_Per_Person'] = data['Fare_Per_Person'].astype(int)

for index,each in enumerate(data['Cabin'].isnull()):
    if each == False:
        data['Cabin'][index] = data['Cabin'][index][0]
data['Cabin'] = data['Cabin'].fillna("ND")
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Has_Cabin,relatives,alone,Age_Class,Fare_Per_Person
0,0,3,Mr,male,2,1,0,0,ND,S,0,1,0,6,0
1,1,1,Mrs,female,5,1,0,3,C,C,1,1,0,5,1
2,1,3,Miss,female,3,0,0,1,ND,S,0,0,1,9,1
3,1,1,Mrs,female,5,1,0,3,C,S,1,1,0,5,1
4,0,3,Mr,male,5,0,0,1,ND,S,0,0,1,15,1


In [1994]:
#
# cat_columns = list(data.select_dtypes(include=['category','object']))
# column_mask = []
# for column_name in list(data_feature_train.columns.values):
#     column_mask.append(column_name in cat_columns)
#col_ohe = list(data_feature_train.columns)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for each in ["Name","Sex","Cabin","Embarked"]:
    data[each] = le.fit_transform(data[each])
data['Age_multi_sex'] = data["Age"]*data["Sex"]
data['Age_multi_Name'] = data["Age"]*data["Name"]
data['Pclass_multi_Name'] =data["Pclass"]*data["Name"]

data['Age_multi_sex_plus_Pclass'] = data['Age_multi_sex']+data['Pclass']
data['Age_multi_sex_plus_Cabin'] = data['Age_multi_sex']+data['Cabin']
data['Pclass_multi_Name_multi_Sex'] = data['Pclass_multi_Name']*data['Sex']
data['Age_multi_sex_multi_Pclass'] = data['Age_multi_sex']*data['Pclass']
data['Cabin_multi_Sex'] = data['Cabin']*data['Sex']
data['Age_multi_sex_multi_cabin'] = data['Age_multi_sex']*data['Cabin']
data['Age_multi_sex_multi_Name'] = data['Age_multi_sex']*data['Name']

#data['modified_Age'] = 
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,...,Age_multi_sex,Age_multi_Name,Pclass_multi_Name,Age_multi_sex_plus_Pclass,Age_multi_sex_plus_Cabin,Pclass_multi_Name_multi_Sex,Age_multi_sex_multi_Pclass,Cabin_multi_Sex,Age_multi_sex_multi_cabin,Age_multi_sex_multi_Name
0,0,3,2,1,2,1,0,0,7,2,...,2,4,6,5,9,6,6,7,14,4
1,1,1,3,0,5,1,0,3,2,0,...,0,15,3,1,2,0,0,0,0,0
2,1,3,1,0,3,0,0,1,7,2,...,0,3,3,3,7,0,0,0,0,0
3,1,1,3,0,5,1,0,3,2,2,...,0,15,3,1,2,0,0,0,0,0
4,0,3,2,1,5,0,0,1,7,2,...,5,10,6,8,12,6,15,7,35,10
5,0,3,2,1,4,0,0,1,7,1,...,4,8,6,7,11,6,12,7,28,8
6,0,1,2,1,6,0,0,3,4,2,...,6,12,2,7,10,2,6,4,24,12
7,0,3,0,1,0,3,1,2,7,2,...,0,0,0,3,7,0,0,7,0,0
8,1,3,3,0,3,0,2,1,7,2,...,0,9,9,3,7,0,0,0,0,0
9,1,2,3,0,1,1,0,2,7,0,...,0,3,6,2,7,0,0,0,0,0


In [1995]:
#data = pd.get_dummies(data)

In [1996]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(data,train_size=0.7)
col = list(data.columns)
col.remove('Survived')
col.remove('relatives')
col.remove('Parch')
col.remove('SibSp')
data_feature_train = data_train[col]
data_label_train = data_train["Survived"]
data_feature_test = data_test[col]
data_label_test = data_test["Survived"]


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [1997]:
# data.nunique()

In [1998]:
data.corr()["Survived"]

Survived                       1.000000
Pclass                        -0.338481
Name                          -0.069046
Sex                           -0.543351
Age                           -0.076961
SibSp                         -0.035322
Parch                          0.081629
Fare                           0.307547
Cabin                         -0.295113
Embarked                      -0.167675
Has_Cabin                      0.316912
relatives                      0.016639
alone                         -0.203367
Age_Class                     -0.306792
Fare_Per_Person                0.181321
Age_multi_sex                 -0.472210
Age_multi_Name                -0.023394
Pclass_multi_Name             -0.312784
Age_multi_sex_plus_Pclass     -0.571523
Age_multi_sex_plus_Cabin      -0.550563
Pclass_multi_Name_multi_Sex   -0.563473
Age_multi_sex_multi_Pclass    -0.510397
Cabin_multi_Sex               -0.558006
Age_multi_sex_multi_cabin     -0.508226
Age_multi_sex_multi_Name      -0.428216


In [1999]:
# data_file.isnull().sum()

In [2000]:
# data.isnull().sum(axis=0)


In [2001]:
# data_feature_train.isnull().sum(axis=0)

# Training and test

In [2002]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(data_feature_train,data_label_train)
pred = rf.predict(data_feature_test)

In [2003]:
from sklearn.metrics import accuracy_score,f1_score
acc = accuracy_score(data_label_test,pred)
fs = f1_score(data_label_test,pred)

In [2004]:
acc

0.832089552238806

In [2005]:
fs

0.7567567567567568

In [2006]:
model = GradientBoostingClassifier()
model.fit(data_feature_train,data_label_train)
#model.fit(x,y)
pred = model.predict(data_feature_test)
print(accuracy_score(data_label_test,pred))
print(data[col].columns)
model.feature_importances_

0.8134328358208955
Index(['Pclass', 'Name', 'Sex', 'Age', 'Fare', 'Cabin', 'Embarked',
       'Has_Cabin', 'alone', 'Age_Class', 'Fare_Per_Person', 'Age_multi_sex',
       'Age_multi_Name', 'Pclass_multi_Name', 'Age_multi_sex_plus_Pclass',
       'Age_multi_sex_plus_Cabin', 'Pclass_multi_Name_multi_Sex',
       'Age_multi_sex_multi_Pclass', 'Cabin_multi_Sex',
       'Age_multi_sex_multi_cabin', 'Age_multi_sex_multi_Name'],
      dtype='object')


array([0.00677762, 0.02126415, 0.00150127, 0.04593753, 0.10422791,
       0.07212222, 0.08988865, 0.00204014, 0.01953225, 0.07941728,
       0.12540291, 0.0036204 , 0.05440174, 0.02335449, 0.13516245,
       0.05228931, 0.05945341, 0.02564641, 0.04820084, 0.02359237,
       0.00616661])

In [2007]:
f1_score(data_label_test,pred)

0.7222222222222222

In [2008]:
# predicting test set
data_file.drop('PassengerId', axis=1, inplace=True)

data_file['Name'] = data_file['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
data_file['Name'] = data_file['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data_file['Name'] = data_file['Name'].replace('Mlle', 'Miss')
data_file['Name'] = data_file['Name'].replace('Ms', 'Miss')
data_file['Name'] = data_file['Name'].replace('Mme', 'Mrs')

data_file['Embarked'] = data_file['Embarked'].fillna("S")

data_file['Has_Cabin'] = data_file["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

data_file['relatives'] = data_file['SibSp'] + data_file['Parch']
data_file.loc[data_file['relatives'] > 0, 'alone'] = 0
data_file.loc[data_file['relatives'] == 0, 'alone'] = 1
data_file['alone'] = data_file['alone'].astype(int)

data_file.drop('Ticket', axis=1, inplace=True)

for index,each in enumerate(data_file["Age"].isnull()):
    if each == True:
        data_file.loc[index,'Age'] = data_file[data_file['Name'] == data_file["Name"][index]]['Age'].mean()
data_file['Age'] = data_file['Age'].astype(int)
data_file.loc[ data_file['Age'] <= 11, 'Age'] = 0
data_file.loc[(data_file['Age'] > 11) & (data_file['Age'] <= 18), 'Age'] = 1
data_file.loc[(data_file['Age'] > 18) & (data_file['Age'] <= 22), 'Age'] = 2
data_file.loc[(data_file['Age'] > 22) & (data_file['Age'] <= 27), 'Age'] = 3
data_file.loc[(data_file['Age'] > 27) & (data_file['Age'] <= 33), 'Age'] = 4
data_file.loc[(data_file['Age'] > 33) & (data_file['Age'] <= 40), 'Age'] = 5
data_file.loc[(data_file['Age'] > 40) & (data_file['Age'] <= 66), 'Age'] = 6
data_file.loc[ data_file['Age'] > 66, 'Age'] = 6

        
        
for index,each in enumerate(data_file['Cabin'].isnull()):
    if each == False:
        data_file['Cabin'][index] = data_file['Cabin'][index][0]
data_file['Cabin'] = data_file['Cabin'].fillna("ND")
data_file['Fare'] = data_file['Fare'].fillna(data_file['Fare'].median())
data_file.loc[ data_file['Fare'] <= 7.91, 'Fare'] = 0
data_file.loc[(data_file['Fare'] > 7.91) & (data_file['Fare'] <= 14.454), 'Fare'] = 1
data_file.loc[(data_file['Fare'] > 14.454) & (data_file['Fare'] <= 31), 'Fare']   = 2
data_file.loc[(data_file['Fare'] > 31) & (data_file['Fare'] <= 99), 'Fare']   = 3
data_file.loc[(data_file['Fare'] > 99) & (data_file['Fare'] <= 250), 'Fare']   = 4
data_file.loc[ data_file['Fare'] > 250, 'Fare'] = 5
data_file['Fare'] = data_file['Fare'].astype(int)

data_file['Age_Class']= data_file['Age']* data_file['Pclass']

data_file['Fare_Per_Person'] = data_file['Fare']/(data_file['relatives']+1)
data_file['Fare_Per_Person'] = data_file['Fare_Per_Person'].astype(int)
    

le = LabelEncoder()
for each in ["Name","Sex","Cabin","Embarked"]:
    data_file[each] = le.fit_transform(data_file[each])

data_file['Age_multi_sex'] = data_file["Age"]*data_file["Sex"]
data_file['Age_multi_Name'] = data_file["Age"]*data_file["Name"]
data_file['Pclass_multi_Name'] = data_file["Pclass"]*data_file["Name"]

data_file['Age_multi_sex_plus_Pclass'] = data_file['Age_multi_sex']+data_file['Pclass']
data_file['Age_multi_sex_plus_Cabin'] = data_file['Age_multi_sex']+data_file['Cabin']
data_file['Pclass_multi_Name_multi_Sex'] = data_file['Pclass_multi_Name']*data_file['Sex']
data_file['Age_multi_sex_multi_Pclass'] = data_file['Age_multi_sex']*data_file['Pclass']
data_file['Cabin_multi_Sex'] = data_file['Cabin']*data_file['Sex']
data_file['Age_multi_sex_multi_cabin'] = data_file['Age_multi_sex']*data_file['Cabin']
data_file['Age_multi_sex_multi_Name'] = data_file['Age_multi_sex']*data_file['Name']


rf = RandomForestClassifier()
rf.fit(data[col],data['Survived'])
pred = rf.predict(data_file[col])
model.score(data[col],data['Survived'])

0.8597081930415263

In [2009]:
# data_file.isnull().sum()

In [2010]:
# f= open("output.csv",'w')
# f.write("PassengerId,Survived\n")
# for ind,each in enumerate(pred):
#     f.write(str(data_result['PassengerId'][ind])+',')
#     f.write(str(each)+'\n')
# f.close()


In [2011]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf,data[col],data['Survived'], cv=10, scoring = "accuracy")
print(scores)
print(scores.mean(), scores.std())

[0.76666667 0.77777778 0.73033708 0.83146067 0.87640449 0.80898876
 0.82022472 0.78651685 0.86516854 0.85227273]
0.8115818295312677 0.04426613018292947


In [2012]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,...,Age_multi_sex,Age_multi_Name,Pclass_multi_Name,Age_multi_sex_plus_Pclass,Age_multi_sex_plus_Cabin,Pclass_multi_Name_multi_Sex,Age_multi_sex_multi_Pclass,Cabin_multi_Sex,Age_multi_sex_multi_cabin,Age_multi_sex_multi_Name
0,0,3,2,1,2,1,0,0,7,2,...,2,4,6,5,9,6,6,7,14,4
1,1,1,3,0,5,1,0,3,2,0,...,0,15,3,1,2,0,0,0,0,0
2,1,3,1,0,3,0,0,1,7,2,...,0,3,3,3,7,0,0,0,0,0
3,1,1,3,0,5,1,0,3,2,2,...,0,15,3,1,2,0,0,0,0,0
4,0,3,2,1,5,0,0,1,7,2,...,5,10,6,8,12,6,15,7,35,10


In [2013]:
# from sklearn.preprocessing import MinMaxScaler
# col_train = data.columns
# col_test = data_file.columns
# minmax = MinMaxScaler()
# data = minmax.fit_transform(data)
# data = pd.DataFrame(data=data,columns=col_train)
# data_file = minmax.fit_transform(data_file)
# data_file = pd.DataFrame(data = data_file,columns = col_test)

In [2014]:
del data["Age_multi_Name"] #
del data['Pclass_multi_Name']
del data["Parch"]
del data["Name"]
del data['Has_Cabin']
del data['alone']
del data['Embarked']
del data['Age_Class']
del data['Fare_Per_Person']
del data['Cabin']
del data['Sex']
del data['Age']
del data["SibSp"]
# del data["Fare"]

data.head()

,Survived,Pclass,Fare,relatives,Age_multi_sex,Age_multi_sex_plus_Pclass,Age_multi_sex_plus_Cabin,Pclass_multi_Name_multi_Sex,Age_multi_sex_multi_Pclass,Cabin_multi_Sex,Age_multi_sex_multi_cabin,Age_multi_sex_multi_Name
0,0,3,0,1,2,5,9,6,6,7,14,4
1,1,1,3,1,0,1,2,0,0,0,0,0
2,1,3,1,0,0,3,7,0,0,0,0,0
3,1,1,3,1,0,1,2,0,0,0,0,0
4,0,3,1,0,5,8,12,6,15,7,35,10


In [2015]:
col = list(data.columns)
col.remove("Survived")
col

['Pclass',
 'Fare',
 'relatives',
 'Age_multi_sex',
 'Age_multi_sex_plus_Pclass',
 'Age_multi_sex_plus_Cabin',
 'Pclass_multi_Name_multi_Sex',
 'Age_multi_sex_multi_Pclass',
 'Cabin_multi_Sex',
 'Age_multi_sex_multi_cabin',
 'Age_multi_sex_multi_Name']

In [2016]:
model = GradientBoostingClassifier()
model.fit(data[col],data['Survived'])
pred = model.predict(data_file[col])
print(data[col].columns)
print(model.feature_importances_)
model.score(data[col],data['Survived'])

Index(['Pclass', 'Fare', 'relatives', 'Age_multi_sex',
       'Age_multi_sex_plus_Pclass', 'Age_multi_sex_plus_Cabin',
       'Pclass_multi_Name_multi_Sex', 'Age_multi_sex_multi_Pclass',
       'Cabin_multi_Sex', 'Age_multi_sex_multi_cabin',
       'Age_multi_sex_multi_Name'],
      dtype='object')
[0.01489898 0.15828101 0.22535374 0.0099561  0.16583045 0.06897811
 0.09145745 0.03082651 0.07513375 0.10854619 0.05073773]


0.8552188552188552

In [2017]:
# gradientboosting
scores = cross_val_score(model,data[col],data['Survived'], cv=10, scoring = "accuracy")
print(scores)
print(scores.mean(), scores.std())

[0.83333333 0.85555556 0.7752809  0.86516854 0.85393258 0.82022472
 0.83146067 0.82022472 0.8988764  0.85227273]
0.8406330155487458 0.031258025191483726


Split dataset

In [2018]:
rf = RandomForestClassifier(criterion= 'gini',min_samples_leaf= 5,min_samples_split = 16,n_estimators = 100,oob_score =True)
rf.fit(data_feature_train,data_label_train)
pred = rf.predict(data_feature_test)
print(data[col].columns)
print(rf.feature_importances_)
print("Oob_Score:",rf.oob_score_)
print(accuracy_score(data_label_test,pred))
f1_score(data_label_test,pred)

Index(['Pclass', 'Fare', 'relatives', 'Age_multi_sex',
       'Age_multi_sex_plus_Pclass', 'Age_multi_sex_plus_Cabin',
       'Pclass_multi_Name_multi_Sex', 'Age_multi_sex_multi_Pclass',
       'Cabin_multi_Sex', 'Age_multi_sex_multi_cabin',
       'Age_multi_sex_multi_Name'],
      dtype='object')
[0.05366956 0.00735803 0.02146665 0.0139231  0.02960941 0.02579459
 0.03219184 0.01067697 0.01137935 0.01823974 0.02312131 0.05950833
 0.01644309 0.02243039 0.19703377 0.05521623 0.06599612 0.08713493
 0.05668901 0.07849734 0.11362023]
Oob_Score: 0.8089887640449438
0.8283582089552238


0.7261904761904762

In [2019]:
rf = RandomForestClassifier(n_estimators=50,oob_score =True)
rf.fit(data[col],data['Survived'])
pred = rf.predict(data_file[col])
print(data[col].columns)
print(rf.feature_importances_)
print("Oob_Score:",rf.oob_score_)
rf.score(data[col],data['Survived'])

Index(['Pclass', 'Fare', 'relatives', 'Age_multi_sex',
       'Age_multi_sex_plus_Pclass', 'Age_multi_sex_plus_Cabin',
       'Pclass_multi_Name_multi_Sex', 'Age_multi_sex_multi_Pclass',
       'Cabin_multi_Sex', 'Age_multi_sex_multi_cabin',
       'Age_multi_sex_multi_Name'],
      dtype='object')
[0.05494205 0.09372804 0.12705373 0.03188477 0.17943967 0.09720683
 0.08941735 0.07982924 0.070391   0.08416826 0.09193907]
Oob_Score: 0.8080808080808081


0.8843995510662177

In [2020]:
# f= open("output.csv",'w')
# f.write("PassengerId,Survived\n")
# for ind,each in enumerate(pred):
#     f.write(str(data_result['PassengerId'][ind])+',')
#     f.write(str(int(each))+'\n')
# f.close()

In [2021]:
scores = cross_val_score(rf,data[col],data['Survived'], cv=10, scoring = "accuracy")
print(scores)
print(scores.mean(), scores.std())

[0.77777778 0.83333333 0.74157303 0.84269663 0.87640449 0.85393258
 0.78651685 0.82022472 0.87640449 0.86363636]
0.8272500283736239 0.04317458602149095


In [2022]:
 for index,each in enumerate(rf.feature_importances_):
    if len(col[index]) <=6:
        sep = "\t\t"
    else:
        sep = "\t"
    print(col[index],"{:.3f}".format(each),sep=sep,)

Pclass		0.055
Fare		0.094
relatives	0.127
Age_multi_sex	0.032
Age_multi_sex_plus_Pclass	0.179
Age_multi_sex_plus_Cabin	0.097
Pclass_multi_Name_multi_Sex	0.089
Age_multi_sex_multi_Pclass	0.080
Cabin_multi_Sex	0.070
Age_multi_sex_multi_cabin	0.084
Age_multi_sex_multi_Name	0.092


In [2023]:
# param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10, 25, 50], "min_samples_split" : [2, 4, 10, 12, 16, 18, 25], "n_estimators": [100, 400, 700]}
# from sklearn.model_selection import GridSearchCV
# rf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
# clf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1)
# clf.fit(data[col], data['Survived'])
# clf.best_params_

In [2024]:
from sklearn.preprocessing import MinMaxScaler
col_train = data.columns
col_test = data_file.columns
minmax = MinMaxScaler()
data = minmax.fit_transform(data)
data = pd.DataFrame(data=data,columns=col_train)
data_file = minmax.fit_transform(data_file)
data_file = pd.DataFrame(data = data_file,columns = col_test)

In [2025]:
rf = RandomForestClassifier(criterion= 'gini',min_samples_leaf= 5,min_samples_split = 16,n_estimators = 100,oob_score =True)
rf.fit(data[col],data['Survived'])
pred = rf.predict(data_file[col])
print(data[col].columns)
print(rf.feature_importances_)
#print(rf.score(data[col],data['Survived']))
print("Oob_Score:",rf.oob_score_)
rf.score(data[col],data['Survived'])

Index(['Pclass', 'Fare', 'relatives', 'Age_multi_sex',
       'Age_multi_sex_plus_Pclass', 'Age_multi_sex_plus_Cabin',
       'Pclass_multi_Name_multi_Sex', 'Age_multi_sex_multi_Pclass',
       'Cabin_multi_Sex', 'Age_multi_sex_multi_cabin',
       'Age_multi_sex_multi_Name'],
      dtype='object')
[0.04710597 0.04146262 0.08100417 0.08098955 0.1833331  0.06217811
 0.13265426 0.06880143 0.0913217  0.11117739 0.09997169]
Oob_Score: 0.8215488215488216


0.8451178451178452

In [2026]:
scores = cross_val_score(rf,data[col],data['Survived'], cv=10, scoring = "accuracy")
print(scores)
print(scores.mean(), scores.std())

[0.8        0.84444444 0.7752809  0.84269663 0.85393258 0.80898876
 0.83146067 0.80898876 0.86516854 0.85227273]
0.8283234025649756 0.027251905212097256


In [2027]:
f= open("output.csv",'w')
f.write("PassengerId,Survived\n")
for ind,each in enumerate(pred):
    f.write(str(data_result['PassengerId'][ind])+',')
    f.write(str(int(each))+'\n')
f.close()

In [2028]:
data.head()

,Survived,Pclass,Fare,relatives,Age_multi_sex,Age_multi_sex_plus_Pclass,Age_multi_sex_plus_Cabin,Pclass_multi_Name_multi_Sex,Age_multi_sex_multi_Pclass,Cabin_multi_Sex,Age_multi_sex_multi_cabin,Age_multi_sex_multi_Name
0,0.0,1.0,0.0,0.1,0.333333,0.500,0.642857,0.75,0.333333,0.875,0.291667,0.166667
1,1.0,0.0,0.6,0.1,0.000000,0.000,0.142857,0.00,0.000000,0.000,0.000000,0.000000
2,1.0,1.0,0.2,0.0,0.000000,0.250,0.500000,0.00,0.000000,0.000,0.000000,0.000000
3,1.0,0.0,0.6,0.1,0.000000,0.000,0.142857,0.00,0.000000,0.000,0.000000,0.000000
4,0.0,1.0,0.2,0.0,0.833333,0.875,0.857143,0.75,0.833333,0.875,0.729167,0.416667


In [2029]:
import matplotlib.pyplot as plt
import seaborn as sns
# data["Age"]*data["relatives"]

In [2030]:
#sns.countplot(x=data["Survived"],hue=data["Pclass"]*data["Name"])

In [2031]:
# import featuretools as ft
# es = ft.EntitySet(id = 'titanic')
# es.entity_from_dataframe(entity_id = 'data', dataframe = data[col], make_index = False,index="index")
# feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data', max_depth = 1,verbose = 1,trans_primitives = ['add_numeric', 'multiply_numeric'])
# #features, feature_names = ft.dfs(entityset=entities, target_entity="Survived", max_depth = 2)
# feature_matrix.head()


In [2032]:
# rf = RandomForestClassifier(criterion= 'gini',min_samples_leaf= 5,min_samples_split = 16,n_estimators = 100,oob_score =True)
# rf.fit(feature_matrix,data['Survived'])
# #pred = rf.predict(data_file[col])
# print(data[col].columns)
# print(rf.feature_importances_)
# print(rf.score(data[col],data['Survived']))
# print("Oob_Score:",rf.oob_score_)
# rf.score(data[col],data['Survived'])

In [2033]:
# col = feature_matrix.columns
# for index,each in enumerate(rf.feature_importances_):
#     if(each > 0.02):
#         if len(col[index]) <=6:
#             sep = "\t\t"
#         else:
#             sep = "\t"
#         print(col[index],"{:.3f}".format(each),sep=sep,)